# 16 - Resource-Aware Optimization

- Prompt Classification : Determines if the user's input is - Uses gpt-4o model with a strict JSON-only response format
    - "simple" – answerable directly
    - "reasoning" – requires logic or multi-step thought
    - "internet_search" – requires up-to-date external information
- If "simple" uses - "gpt-4o-mini"
- If "reasoning" uses - "o4-mini"
- If "internet search" - call Google Search API and then passes response to "gpt-4o" 


In [28]:
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI


# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_CUSTOM_SEARCH_API_KEY = os.getenv("GOOGLE_CUSTOM_SEARCH_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

if not OPENAI_API_KEY or not GOOGLE_CUSTOM_SEARCH_API_KEY or not GOOGLE_CSE_ID:
    raise ValueError(
        "Please set OPENAI_API_KEY, GOOGLE_CUSTOM_SEARCH_API_KEY, and GOOGLE_CSE_ID in your .env file."
    )

client = OpenAI(api_key=OPENAI_API_KEY)


# --- Step 1: Classify the Prompt ---
def classify_prompt(prompt: str) -> dict:
    system_message = {
        "role": "system",
        "content": (
            "You are a classifier that analyzes user prompts and returns one of three categories ONLY:\n\n"
            "- simple\n"
            "- reasoning\n"
            "- internet_search\n\n"
            "Rules:\n"
            "- Use 'simple' for direct factual questions that need no reasoning or current events.\n"
            "- Use 'reasoning' for logic, math, or multi-step inference questions.\n"
            "- Use 'internet_search' if the prompt refers to current events, recent data, or things not in your training data.\n\n"
            "Respond ONLY with JSON like:\n"
            '{ "classification": "simple" }'
        ),
    }

    user_message = {"role": "user", "content": prompt}

    response = client.chat.completions.create(
        model="gpt-4o", messages=[system_message, user_message], temperature=1
    )

    reply = response.choices[0].message.content
    return json.loads(reply)


# --- Step 2: Google Search ---
def google_search(query: str, num_results=1) -> list:
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": GOOGLE_CUSTOM_SEARCH_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "q": query,
        "num": num_results,
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        results = response.json()

        if "items" in results and results["items"]:
            return [
                {
                    "title": item.get("title"),
                    "snippet": item.get("snippet"),
                    "link": item.get("link"),
                }
                for item in results["items"]
            ]
        else:
            return []
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}


# --- Step 3: Generate Response ---
def generate_response(prompt: str, classification: str, search_results=None) -> str:
    if classification == "simple":
        model = "gpt-4o-mini"
        full_prompt = prompt
    elif classification == "reasoning":
        model = "o4-mini"
        full_prompt = prompt
    elif classification == "internet_search":
        model = "gpt-4o"
        # Convert each search result dict to a readable string
        if search_results:
            search_context = "\n".join(
                [
                    f"Title: {item.get('title')}\nSnippet: {item.get('snippet')}\nLink: {item.get('link')}"
                    for item in search_results
                ]
            )
        else:
            search_context = "No search results found."
        full_prompt = f"""Use the following web results to answer the user query:

{search_context}

Query: {prompt}"""

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": full_prompt}],
        temperature=1,
    )

    return response.choices[0].message.content, model


# --- Step 4: Combined Router ---
def handle_prompt(prompt: str) -> dict:
    classification_result = classify_prompt(prompt)
    # Remove or comment out the next line to avoid duplicate printing
    # print("\n🔍 Classification Result:", classification_result)
    classification = classification_result["classification"]

    search_results = None
    if classification == "internet_search":
        search_results = google_search(prompt)
        # print("\n🔍 Search Results:", search_results)

    answer, model = generate_response(prompt, classification, search_results)
    return {"classification": classification, "response": answer, "model": model}

In [29]:
test_prompt = "What is the capital of Australia?"
# test_prompt = "Explain the impact of quantum computing on cryptography."
# test_prompt = "When does the Australian Open 2026 start, give me full date?"

result = handle_prompt(test_prompt)
print("🔍 Classification:", result["classification"])
print("🧠 Model Used:", result["model"])
print("🧠 Response:\n", result["response"])

🔍 Classification: simple
🧠 Model Used: gpt-4o-mini
🧠 Response:
 The capital of Australia is Canberra.


In [32]:
# test_prompt = "What is the capital of Australia?"
test_prompt = "When does the Australian Open 2026 start, give me full date with year?"
# test_prompt = "Explain the impact of quantum computing on cryptography."

result = handle_prompt(test_prompt)
print("🔍 Classification:", result["classification"])
print("🧠 Model Used:", result["model"])
print("🧠 Response:\n", result["response"])

🔍 Classification: internet_search
🧠 Model Used: gpt-4o
🧠 Response:
 The Australian Open 2026 starts on January 19, 2026.


In [33]:
# test_prompt = "What is the capital of Australia?"
# test_prompt = "When does the Australian Open 2026 start, give me full date?"
test_prompt = "Explain the impact of quantum computing on cryptography."

result = handle_prompt(test_prompt)
print("🔍 Classification:", result["classification"])
print("🧠 Model Used:", result["model"])
print("🧠 Response:\n", result["response"])

🔍 Classification: reasoning
🧠 Model Used: o4-mini
🧠 Response:
 Quantum computers change the rules of the game in two major ways: by giving powerful new algorithms that can break many of today’s public-key schemes, and by offering novel cryptographic primitives (like quantum key distribution) for new security models. Below is a high-level overview of the impact and the community’s response.  

1. Shor’s Algorithm and the Threat to Public-Key Cryptography  
  • Shor’s Algorithm (1994): in principle, a sufficiently large, fault-tolerant quantum computer can factor large integers and compute discrete logarithms in polynomial time.  
  • Broken Schemes:  
    – RSA (security relies on integer factoring)  
    – Diffie-Hellman and ECDH (security relies on discrete logs in finite fields or elliptic curves)  
    – DSA, ECDSA, ElGamal, many pairing-based protocols  
  • Real-World Consequence: any data encrypted today under these schemes, if captured and stored, could be decrypted once a quant